In [1]:
!pip install pydicom
!pip install nibabel
!pip install dcm2niix
!pip install nipype
# !git clone https://github.com/ANTsX/ANTsPy
# !python3 setup.py install
!pip install antspyx


[notice] A new release of pip is available: 23.3 -> 23.3.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.3 -> 23.3.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.3 -> 23.3.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.3 -> 23.3.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.3 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [2]:
import os
import pydicom
import nibabel as nib
import numpy as np
import subprocess
from scipy.ndimage import zoom
import shutil
import subprocess
import ants
from nipype.interfaces.fsl import BET

In [3]:
dataset_root = './dataset/ADNI_FINAL_DATASET'

## Folder Creation

In [3]:
def remove_ds_store_files(directory):
    for root, dirs, files in os.walk(directory):
        for filename in files:
            if filename == ".DS_Store":
                file_path = os.path.join(root, filename)
                os.remove(file_path)

In [5]:
# # Do not run

# def convert_dicom_to_nifti(input_folder, output_folder):
#     cmd = ["dcm2niix", "-z", "y", "-o", output_folder, input_folder]
#     subprocess.run(cmd)

# def process_patient_folder(patient_id):
#     patient_path = os.path.join(dataset_root, patient_id)
#     for subfolder in os.listdir(patient_path):
#         subfolder_path = os.path.join(patient_path, subfolder)
#         for date in os.listdir(subfolder_path):
#             date_path = os.path.join(subfolder_path, date)
#             image_id = os.listdir(date_path)[0]
#             images_path = os.path.join(date_path, image_id)
#             output_path = os.path.join("./dataset/ADNI_PREPROCESSED", patient_id, image_id)
#             os.makedirs(output_path, exist_ok=True)
#             convert_dicom_to_nifti(images_path, output_path)


# remove_ds_store_files(dataset_root)
    
# for patient_id in os.listdir(dataset_root):
#     process_patient_folder(patient_id)

# remove_ds_store_files(dataset_root)


## Preprocessing of MRI Images

#### 1. Reorientation using fslreorient2std
#### 2. Bias Field Correction using ANTs
#### 3. MNI152 Registration
#### 3. Skull Stripping using BET
#### 4. Extraction of most important slices of brain volume

In [4]:
def reorientation(input_path,output_path):
    subprocess.run(['/Users/swethathomas/fsl/bin/fslreorient2std', input_path, output_path])

In [5]:
def bias_correction(input_path,output_path):
    image = ants.image_read(input_path)
    bias_corrected_image = ants.n4_bias_field_correction(image)
    ants.image_write(bias_corrected_image, output_path)

In [6]:
def MNI152_registration(input_path, output_path):
    reg_command = [
    'flirt',
    '-in', input_path,
    '-ref', './dataset/templates/MNI152_T1_1mm.nii.gz',
    '-out', output_path
    ]
    subprocess.run(reg_command)

In [7]:
def skull_stripping(input_path, output_path):
    skullstrip = BET()
    skullstrip.inputs.in_file = input_path
    skullstrip.inputs.out_file = output_path
    skullstrip.run()

In [10]:
def extract_slices(input_path, output_path):
    def get_middle_slices(nii_image, num_slices):
        image_data = nii_image.get_fdata()
        num_total_slices = image_data.shape[0]
        start_slice = num_total_slices // 2 - num_slices // 2
        end_slice = start_slice + num_slices
        return image_data[start_slice:end_slice,:,:]

    def resize_image(image):
        resized_image = zoom(image, (110/image.shape[0], 110/image.shape[1]), order=3)
        return resized_image

    nii_image = nib.load(input_path)
    middle_slices = get_middle_slices(nii_image, 110)

    resized_slices = np.empty((110, 110, 110))
    for i in range(110):
        resized_slices[i] = resize_image(middle_slices[i])
    resized_nii_image = nib.Nifti1Image(resized_slices, affine=nii_image.affine)
    nib.save(resized_nii_image, output_path)

In [8]:
def preprocess_mri(input_path, output_path):
    # Reorientation
    reorientation(input_path, output_path)

    # 2. Bias field correction
    bias_correction(output_path,output_path)

    # 3. MN152 Registration
    MNI152_registration(output_path, output_path)

    # 4. Skull stripping
    skull_stripping(output_path, output_path)

    # # 5. Extraction of most important slices
    # extract_slices(output_path, output_path)

In [10]:
dataset_root = "./dataset/ADNI"
remove_ds_store_files(dataset_root)

count = 0
for patient_folder in os.listdir(dataset_root):
    count+=1
    if count==20:
        break
    elif count>=10:
        patient_folder_path = os.path.join(dataset_root, patient_folder)    
        for image_folder in os.listdir(patient_folder_path):
            image_folder_path = os.path.join(patient_folder_path, image_folder)
            if not os.path.exists(os.path.join('./dataset/ADNI_PREPROCESSED_GRAD', patient_folder, image_folder)):
                os.makedirs(os.path.join('./dataset/ADNI_PREPROCESSED_GRAD', patient_folder, image_folder), exist_ok=True)
                for image in os.listdir(image_folder_path):
                    input_path = os.path.join(image_folder_path, image)
                    output_path = os.path.join('./dataset/ADNI_PREPROCESSED_GRAD', patient_folder, image_folder, image)
                    if input_path.endswith('.nii.gz'):  
                        preprocess_mri(input_path, output_path)



Final result: 
1.024189 0.019795 0.017657 -15.955554 
-0.022513 1.037810 0.049958 -7.028512 
-0.008750 -0.061094 1.046072 -52.331073 
0.000000 0.000000 0.000000 1.000000 


Final result: 
1.012681 -0.040024 0.097227 -17.579790 
0.016165 1.033977 0.104945 -13.484584 
-0.102624 -0.118171 1.037155 -31.626403 
0.000000 0.000000 0.000000 1.000000 


Final result: 
1.013241 0.013052 0.013394 -12.566737 
-0.024189 1.052395 0.042493 -7.444893 
-0.009570 -0.066043 1.064239 -40.499934 
0.000000 0.000000 0.000000 1.000000 


Final result: 
1.007244 -0.040057 0.084318 -15.826334 
0.023982 1.033223 0.086492 -11.433677 
-0.087542 -0.088134 1.064241 -37.224245 
0.000000 0.000000 0.000000 1.000000 


Final result: 
1.043826 0.053184 0.056101 -24.478044 
-0.058880 1.037532 0.051521 2.811033 
-0.045785 -0.070543 1.049340 -45.104651 
0.000000 0.000000 0.000000 1.000000 


Final result: 
1.096325 0.029968 0.004790 -17.445962 
-0.022262 0.978109 0.568969 -66.534411 
0.018107 -0.638927 1.004676 10.987835 
